Reload .py files whenever there are changes

In [1]:
%load_ext autoreload
%autoreload 2

Local imports from this project

In [2]:
from mdf import *
from mdf.ops import *
from mdf.data import *

External imports from dependencies

In [3]:
from pytorch3d.ops.laplacian_matrices import norm_laplacian, cot_laplacian, laplacian
from pytorch3d.ops import norm_laplacian, cot_laplacian, laplacian
from pyvista.plotting.plotter import Plotter
import plotly.graph_objects as go
import plotly.express as px
from torch import Tensor
from typing import cast
import torch_geometric
import torch.nn as nn
import pyvista as pv
import scipy.linalg
import torch.linalg
import numpy as np
import pytorch3d
import einops
import torch

Environment setup

In [4]:
pv.set_jupyter_backend('trame')

Data Loading

In [5]:
# Download standard shapes
data = DataManager(data_dir=data_dir, cache_dir=cache_dir)

How to apply a texture to a 3d mesh

In [6]:
vista  = data.objects['stanford-bunny'].vista
texture = pv.Texture(data.weather[42]['data'])
vista.texture_map_to_sphere(inplace=True)
vista.plot(texture=texture)

/home/invokariman/.cache/pypoetry/virtualenvs/mdf-B-9uJKhD-py3.11/lib/python3.11/site-packages/pytorch3d/io/obj_io.py:546: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


Widget(value='<iframe src="http://localhost:32955/index.html?ui=P_0x7d0a5b170bd0_0&reconnect=auto" class="pyvi…

Compute positional embedding using Laplace-Beltrami Operator

In [15]:
verts = torch.tensor(vista.points)
faces = torch.tensor(vista.faces)
lbo = LBOEmbedding.from_poly(vista, k_evecs=10, device=torch.device('cuda'))

In [16]:
index = torch.tensor([[0, 0, 0]])
index

bary = torch.tensor([[1/3, 1/3, 1/3]])
bary

tensor([[0.3333, 0.3333, 0.3333]])

In [21]:
verts_idx = einops.repeat(torch.arange(lbo.verts.size(0)), 'N -> N 3')
verts_idx

tensor([[    0,     0,     0],
        [    1,     1,     1],
        [    2,     2,     2],
        ...,
        [34831, 34831, 34831],
        [34832, 34832, 34832],
        [34833, 34833, 34833]])

In [22]:
x = lbo(verts_idx)

torch.Size([3, 34834, 10]) torch.Size([34834, 3, 10])


RuntimeError: Size does not match at dimension 0 expected index [34834, 3, 10] to be smaller than self [3, 34834, 10] apart from dimension 1

In [ ]:
# Find the smallest k eigenvalues and their respective eigenvectors
k_eigenvals = 10
val, vec = torch.lobpcg(L.cuda(), k=k_eigenvals, largest=False)

def eigenfunction(vertex_index: int, eigenvector: torch.Tensor) -> torch.Tensor:
    return eigenvector[vertex_index] * torch.sqrt(torch.tensor(eigenvector.size(0)))

def eigen(vertex_index: int, eigenvectors: torch.Tensor) -> torch.Tensor:
    return eigenvectors[vertex_index, :].sum() * torch.sqrt(torch.tensor(eigenvectors.size(0)))

vec = torch.nn.functional.normalize(vec, dim=0)
vec = vec.cpu()

In [ ]:
x = np.array([eigen(i, vec[:, 1:2]) for i in range(verts.size(0))])
pe = pv.PolyData(verts_inuse.numpy(), faces_inuse)
p = Plotter()
p.add_mesh(pe, scalars=x, clim=[-1, 1], cmap='RdBu_r')
p.show()